<a href="https://colab.research.google.com/github/MKJJang/airflow/blob/master/03_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 허깅페이스 트랜스포머 활용에 필요한 라이브러리 설치
!pip install transformers==4.40.1 datasets==2.19.0 huggingface_hub==0.23.0 -qqq

In [7]:
#3개 라이브러리(transformers, datasets, huggingface_hub) 설치 후 버전 확인
#설치 완료 후, Huggging Face 관련 작업, 예를 들어 사전 훈련된 언어 모델을 활용한 작업을 수행할 수 있음.
## transformers 토크나이저 활용할때 사용
## datasets데이터셋 공개하고 쉽게 가져다 쓸 수 있도록 지원하여 트랜스포머 모델을 쉽게 학습하고 추론에 활용할 수 있도록 돕는다.
import transformers
import datasets
import huggingface_hub

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)

4.40.1
2.19.0
0.23.0


3.1 BERT와 GPT-2 모델을 활용할 때 허깅페이스 트랜스포머 코드 비교

In [13]:
from transformers import AutoModel, AutoTokenizer

text = "What is Huggingface Transformers?"
# BERT 모델 활용
bert_model = AutoModel.from_pretrained("bert-base-uncased")
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
encoded_input = bert_tokenizer(text, return_tensors='pt')
bert_output = bert_model(**encoded_input)

# GPT-2 모델 활용
gpt_model = AutoModel.from_pretrained('gpt2')
gpt_tokenizer = AutoTokenizer.from_pretrained('gpt2')
encoded_input = gpt_tokenizer(text, return_tensors='pt')
gpt_output = gpt_model(**encoded_input)

In [14]:
print("BERT Output Shape:", bert_output.last_hidden_state.shape)  # BERT 출력 확인

BERT Output Shape: torch.Size([1, 8, 768])


3.2 모델 아이디로 모델 불러오기 (모델 바디 불러오기)

In [26]:
from transformers import AutoModel
model_id = 'klue/roberta-base'  # RoBERTa 는 구글의 BERT를 개선한 모델이다.
model = AutoModel.from_pretrained(model_id)

# AutoModel 클래스는 어떻게 klue/roverta-base 저장소의 모델이 RoBERTa 계열의 모델인지 알 수 있을까?
# 허깅페이스는 모델을 저장할 때 congif.json 파일이 함께 저장되는데, 해달 설정 파일에는 아래와 같이 모델 종류 및 파라미터, 클래스 등이 저장되어 확인할 수 있다.
# 아래 3.3 AutoModel에 Model이 roberta로 되어 있고, AutoModel은 이 정보를 통해 RoBERTa 모델을 불러온다.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# 3.3 klue/robera-base 모델의 config.json 파일의 일부
from transformers  import AutoConfig

config = AutoConfig.from_pretrained(model_id)
print(config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.40.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}



3.4 모델 헤드가 포함된 모델 불러오기 (텍스트 분류 헤드가 붙어 있는 모델)